### Reading the data

In this notebook we will inspect the digitized data, both data and MC, and also the geometry.
Lets start off with a data file. We will be using ROOT. 

The path+filename is: /eos/experiment/sndlhc/convertedData/physics/2022/run_004705/sndsw_raw-0010.root 

In [1]:
import ROOT

Welcome to JupyROOT 6.28/04


In [2]:
file = ROOT.TFile("/eos/experiment/sndlhc/convertedData/physics/2022/run_004705/sndsw_raw-0010.root")
tree = file.rawConv
tree.GetEntries()

1000000

To print the branch content of the TTree:

In [3]:
tree.Print()

******************************************************************************
*Tree    :rawConv   : /cbmout_0                                              *
*Entries :  1000000 : Total =      5068918144 bytes  File  Size =  783658867 *
*        :          : Tree compression factor =   6.47                       *
******************************************************************************
*Branch  :EventHeader. : cbmout_0.sndEventHeader.EventHeader.                   *
*Entries :  1000000 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :EventHeader.TNamed.fUniqueID :                                     *
*         | UInt_t cbmout_0.sndEventHeader.EventHeader.TNamed.fUniqueID      *
*Entries :  1000000 : Total  Size=    4007376 bytes  File Size  =      25085 *
*Baskets :       59 : Basket Size=      94720 bytes  Compression= 159.69     *
*................................................

In [4]:
tree.GetEvent(0) # get the first event in the file - this command reads the index in the file
tree.EventHeader.GetEventNumber()

10000000

Remember we opened partition number 10, so the 1st event there is actually the 10th M event for the run - the event counter doesn't reset with partitions! 
The index in the ROOT file however, does reset.

In [5]:
tree.EventHeader.GetFillNumber()

8088

The fill number can be used to check out fill information from the SND run DB: https://sndrundb.cern.ch/app/ or the LHC supertables: https://bpt.web.cern.ch/lhc/supertable/2023/static_lhc_supertable.html

In [6]:
tree.GetEvent(7)
tree.EventHeader.GetEventNumber()

10000007

In [7]:
tree.EventHeader.GetUTCtimestamp()

1659579405

In [8]:
tree.EventHeader.GetTimeAsString()

'Thu Aug  4 02:16:45 2022\n'

In [9]:
tree.Digi_MuFilterHits.GetEntries()

16

In [11]:
tree.Digi_MuFilterHits.Dump()

==> Dumping object at: 0x0000000013adf130, name=MuFilterHit, class=MuFilterHit

flag                          1                   flag
fMasked[16]                   0                   / masked signal ...
fDetectorID                   10002               Detector unique identifier
nSiPMs                        8                   / number of SiPMs per side
nSides                        2                   / number of sides
signals[16]                   32.89               / SiPM signal ...
times[16]                     0.614               / SiPM time ...
fDaqID[16]                    9158761             / encodes rawhitindex*100000+(board_id * 1000) + (tofpet_id * 100) + tofpet_channel ...
fUniqueID                     0                   object unique identifier
fBits                         0x03000000          bit field status word
==> Dumping object at: 0x0000000013adf260, name=MuFilterHit, class=MuFilterHit

flag                          1                   flag
fMasked[16]        

Get the MuFi system, plane and orientation.

system name = numbering in GetSystem()

### Veto = 1

### US = 2

### DS = 3

We will have to work a bit more to get the bar number :-D

In [12]:
for aHit in tree.Digi_MuFilterHits: # loop over hits
    print(aHit.GetSystem(), aHit.GetDetectorID(), aHit.GetPlane(), aHit.isVertical(), aHit.GetDetectorID()%1000)

1 10002 0 False 2
1 11002 1 False 2
2 20003 0 False 3
2 21003 1 False 3
2 22003 2 False 3
2 23003 3 False 3
2 24003 4 False 3
3 30023 0 False 23
3 30024 0 False 24
3 30089 0 True 89
3 31024 1 False 24
3 31088 1 True 88
3 31089 1 True 89
3 32024 2 False 24
3 32085 2 True 85
3 33085 3 True 85


The plane numbering in the sndsw starts from 0! So does the bar count!


Lets read out the hardware information from the hits - the tofpet_id, tofpet_channel, the board_id, etc.

In [14]:
for aHit in tree.Digi_MuFilterHits: # loop over MuFilter hits
    # loop over all channels per bar
    for side in range(aHit.GetnSides()): # loop over sides - left/right or top
        for channel in range(aHit.GetnSiPMs()): # loop over channels per side
            ch = 8*side + channel
            #print(aHit.GetBoardID(ch), aHit.GetTofpetID(ch), aHit.Getchannel(ch), aHit.GetSystem(), aHit.GetDetectorID(),side, channel)
            print(aHit.GetBoardID(ch), aHit.GetTofpetID(ch), aHit.Getchannel(ch), round(aHit.GetSignal(ch),2), aHit.GetSystem(), aHit.GetDetectorID(),aHit.isVertical(), side, channel)

58 7 61 32.89 1 10002 False 0 0
58 7 57 36.53 1 10002 False 0 1
0 0 -1 -999.0 1 10002 False 0 2
58 7 49 32.39 1 10002 False 0 3
58 7 45 26.22 1 10002 False 0 4
58 7 41 27.87 1 10002 False 0 5
0 0 -1 -999.0 1 10002 False 0 6
58 7 33 22.0 1 10002 False 0 7
58 1 61 35.87 1 10002 False 1 0
58 1 57 30.5 1 10002 False 1 1
0 0 -1 -999.0 1 10002 False 1 2
58 1 49 39.15 1 10002 False 1 3
58 1 45 31.47 1 10002 False 1 4
58 1 41 37.14 1 10002 False 1 5
58 1 37 33.93 1 10002 False 1 6
58 1 33 38.97 1 10002 False 1 7
58 5 61 13.63 1 11002 False 0 0
58 5 57 16.9 1 11002 False 0 1
58 5 53 13.85 1 11002 False 0 2
58 5 49 14.83 1 11002 False 0 3
58 5 45 14.89 1 11002 False 0 4
58 5 41 16.61 1 11002 False 0 5
58 5 37 13.35 1 11002 False 0 6
58 5 33 10.79 1 11002 False 0 7
58 3 61 15.17 1 11002 False 1 0
58 3 57 17.9 1 11002 False 1 1
58 3 53 18.63 1 11002 False 1 2
58 3 49 15.89 1 11002 False 1 3
58 3 45 15.48 1 11002 False 1 4
58 3 41 14.28 1 11002 False 1 5
58 3 37 18.66 1 11002 False 1 6
58 3 33 14.3

The channel numbering in the sndsw starts from 0!
The default value for QDC per channel is -999! What happens is: if at least 1 SiPM fires, a hit is created with all other channels assigned QDC=-999. 
If another SiPM of the same bar fires, its QDC is recorded in place of the default -999.
In TI18 data most recorded events are passing muons and the **small** SiPMs are unlikely to fire.

Lets read out some SciFi hits info:

In [16]:
for aHit in tree.Digi_ScifiHits: # loop over SciFi hits
    print(aHit.GetStation(), aHit.GetMat(), aHit.GetSiPM(), aHit.GetSiPMChan(), aHit.GetChannelID(), aHit.GetDetectorID(), round(aHit.GetSignal(),2), aHit.GetTime())

1 1 1 4 1011004 1011004 4.72 2.003999948501587
1 1 1 5 1011005 1011005 -5.23 3.3910000324249268
1 1 1 14 1011014 1011014 -3.06 2.4730000495910645
1 1 1 15 1011015 1011015 -2.48 2.1679999828338623
1 1 1 16 1011016 1011016 -0.4 1.9010000228881836
1 1 1 17 1011017 1011017 -6.45 1.9509999752044678
1 1 1 18 1011018 1011018 -0.43 1.9859999418258667
1 1 1 19 1011019 1011019 1.43 3.7260000705718994
1 1 1 21 1011021 1011021 0.1 2.3380000591278076
1 1 1 22 1011022 1011022 -2.32 2.0179998874664307
1 1 1 112 1111112 1111112 4.59 1.812999963760376
1 1 1 113 1111113 1111113 2.7 1.8519999980926514
2 1 1 14 2011014 2011014 -1.27 2.0380001068115234
2 1 1 15 2011015 2011015 4.01 3.2079999446868896
2 1 1 112 2111112 2111112 -1.49 2.122999906539917
2 1 1 113 2111113 2111113 1.46 1.9789999723434448
2 1 1 114 2111114 2111114 -15.11 2.2780001163482666
3 1 1 22 3011022 3011022 2.92 1.9079999923706055
3 1 1 114 3111114 3111114 -2.03 1.9800000190734863
3 1 1 115 3111115 3111115 -1.51 2.255000114440918
4 1 1 30 

 Here comes our pain: negative QDC per MIP (mostly muons in TI18 data). In the data, we store the timestamp in clock cycles.

### Reading the MC

 The path+filename is: /eos/experiment/sndlhc/MonteCarlo/MuonBackground/muons_down/scoring_1.8_Bfield/sndLHC.Ntuple-TGeant4-160urad_magfield_2022TCL6_muons_rock_5e7pr_digCPP.root

In [17]:
file_mc = ROOT.TFile("/eos/experiment/sndlhc/MonteCarlo/MuonBackground/muons_down/scoring_1.8_Bfield/sndLHC.Ntuple-TGeant4-160urad_magfield_2022TCL6_muons_rock_5e7pr_digCPP.root")
tree_mc = file_mc.cbmsim
tree_mc.GetEntries()

386629

In [18]:
tree_mc.Print()

******************************************************************************
*Tree    :cbmsim    : /cbmout_0                                              *
*Entries :   386629 : Total =      4342454019 bytes  File  Size = 1766650009 *
*        :          : Tree compression factor =   2.46                       *
******************************************************************************
*Br    0 :MCTrack   : Int_t cbmout_0.ShipMCTrack.MCTrack_                    *
*Entries :   386629 : Total  Size=    3182000 bytes  File Size  =    1223519 *
*Baskets :      183 : Basket Size=      32000 bytes  Compression=   2.54     *
*............................................................................*
*Br    1 :MCTrack.fUniqueID : UInt_t fUniqueID[cbmout_0.ShipMCTrack.MCTrack_]*
*Entries :   386629 : Total  Size=  253205688 bytes  File Size  =    2231927 *
*Baskets :      195 : Basket Size=    4435456 bytes  Compression= 113.45     *
*...................................................

In [20]:
tree_mc.GetEvent(0) # get event at index 0
tree_mc.EventHeader.GetEventNumber() # get its event number

1

### There is a #1 mismatch btw index and event number in the MC!

In [21]:
tree_mc.GetEvent(90)
# make the output easier to grasp - decoration is important!
from decorators import *
tree_mc.MCTrack.Dump()

0 ("ShipMCTrack") pdgCode:     -13(       mu+)  Z= -61.3 m P=197.956 GeV/c mother=-1 Bending in magnetic field
1 ("ShipMCTrack") pdgCode:      11(        e-)  Z=   5.1 m P= 5.385 GeV/c mother=0 Delta ray
2 ("ShipMCTrack") pdgCode:      22(     gamma)  Z=   5.1 m P= 0.307 GeV/c mother=1 Bremstrahlung
3 ("ShipMCTrack") pdgCode:     -11(        e+)  Z=   5.1 m P= 0.135 GeV/c mother=2 Lepton pair production
4 ("ShipMCTrack") pdgCode:      22(     gamma)  Z=   5.1 m P= 0.106 GeV/c mother=3 Bremstrahlung
5 ("ShipMCTrack") pdgCode:      11(        e-)  Z=   5.1 m P= 0.172 GeV/c mother=2 Lepton pair production
6 ("ShipMCTrack") pdgCode:      22(     gamma)  Z=   5.1 m P= 0.137 GeV/c mother=5 Bremstrahlung
7 ("ShipMCTrack") pdgCode:      22(     gamma)  Z=   5.1 m P= 1.084 GeV/c mother=1 Bremstrahlung
8 ("ShipMCTrack") pdgCode:     -11(        e+)  Z=   5.1 m P= 0.776 GeV/c mother=7 Lepton pair production
9 ("ShipMCTrack") pdgCode:      22(     gamma)  Z=   5.1 m P= 0.198 GeV/c mother=8 Bremstr

 Lets inspect the MC Points:

In [22]:
for aPoint in tree_mc.ScifiPoint:
    aPoint.Dump() # one entry at a time so no beautifying

==> Dumping object at: 0x000000002657b3c0, name=ScifiPoint, class=ScifiPoint

fPdgCode                      -13                 
fTrackID                      0                   Track index
fEventId                      0                   MC Event id
fPx                           -0.238762           Momentum components [GeV]
fPy                           -0.102129           Momentum components [GeV]
fPz                           165.075             Momentum components [GeV]
fTime                         1611.05             Time since event start [ns]
fLength                       6433.07             Track length since creation [cm]
fELoss                        1.89756e-05         Energy loss at this point [GeV]
fDetectorID                   1031218             Detector unique identifier
fX                            -43.2514            Position of hit [cm]
fY                            46.8529             Position of hit [cm]
fZ                            300.225             Positio

Lets check out the energy loss in preferred units:

In [23]:
import shipunit as u # the units module
for aPoint in tree_mc.ScifiPoint:
    if aPoint.GetTrackID() == -2:
        print(aPoint.GetEnergyLoss()/u.keV, aPoint.GetEnergyLoss()/u.eV) # printing in [keV] and [eV] for comparison

13.840890460414812 13840.890460414812
75.3018757677637 75301.8757677637
34.01539652259089 34015.39652259089
45.22964445641264 45229.64445641264
26.458310458110645 26458.310458110645
38.25246312771924 38252.46312771924
38.31136564258486 38311.36564258486
2.6715522380982293 2671.5522380982293
30.548970244126394 30548.97024412639
25.6528928730404 25652.8928730404
30.23737735929899 30237.377359298986
27.746289561036974 27746.289561036974
24.186259906855412 24186.259906855412
35.622222640085965 35622.222640085965
25.70064134488348 25700.64134488348
28.9899453491671 28989.9453491671
6.546474196511554 6546.474196511554
35.700391890713945 35700.391890713945
17.363086953992024 17363.086953992024
53.83048846852034 53830.48846852034
11.098091817984823 11098.091817984823
30.417551897699013 30417.55189769901
85.60454443795606 85604.54443795606
52.29056841926649 52290.56841926649
53.499108616961166 53499.108616961166


#### Particles below a preset Ecut do not have associated MCTracks (TrackID=-2 is dummy), but these particles are still propagated and their MCPoints recorded thus used to make Digi_hits
Lets check if that is true!
To do so, we will access the list of MC points used to generate digitized hits.

In [24]:
istrue = False
for aHit in tree_mc.Digi_ScifiHits: # loop over digi hits
    # loop over mc points used to make the digi hit using the unique ID == detID
    for mc_point_i, _ in tree_mc.Digi_ScifiHits2MCPoints[0].wList(aHit.GetDetectorID()) : 
            #ask if the track id for the point is -2   
            if tree_mc.ScifiPoint[mc_point_i].GetTrackID()==-2:
                istrue = True
                break
    if istrue:
        break
print(istrue)    

True


Use the primary track (index is 0) in an event to get the statistical weight of the full event:

In [31]:
tree_mc.GetEvent(90)
print(tree_mc.MCTrack[0].GetWeight())
print('Another event now')
tree_mc.GetEvent(2493)
print(tree_mc.MCTrack[0].GetWeight())

0.009974133223295212
Another event now
0.0013000000035390258


### Lets read some geofile
The geofile + path is /eos/experiment/sndlhc/convertedData/physics/2023/geofile_sndlhc_TI18_V3_2023.root

In [32]:
import SndlhcGeo
geo = SndlhcGeo.GeoInterface("/eos/experiment/sndlhc/convertedData/physics/2023/geofile_sndlhc_TI18_V3_2023.root")
Scifi = geo.modules['Scifi']
MuFi = geo.modules['MuFilter']

using geofile: root://eospublic.cern.ch//eos/experiment/sndlhc/convertedData/physics/2023/geofile_sndlhc_TI18_V3_2023.root


Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 7, max placements = 2829
Info in <TGeoManager::CloseGeometry>: 342536 nodes/ 81 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------


In [33]:
Scifi.Dump()

==> Dumping object at: 0x0000000026f59660, name=Scifi, class=Scifi

fTrackID                      -1                  !  track index
fVolumeID                     -1                  !  volume id
fPos                          ->26f59728          !  position at entrance
fPos.fP                       ->26f59738          3 vector component
fPos.fP.fX                    0                   
fPos.fP.fY                    0                   
fPos.fP.fZ                    0                   
fPos.fP.fUniqueID             0                   object unique identifier
fPos.fP.fBits                 0x03000000          bit field status word
fPos.fE                       0                   time or energy of (x,y,z,t) or (px,py,pz,e)
fPos.fUniqueID                0                   object unique identifier
fPos.fBits                    0x03000000          bit field status word
fMom                          ->26f59768          !  momentum at entrance
fMom.fP                       ->26f59778      

Reading the geo parameters: example reading the speed of light in the fibers

In [34]:
Scifi.GetConfParF('Scifi/signalSpeed') # reading the detector object

15.0

In [35]:
geo.snd_geo['Scifi']['signalSpeed'] # reading the geo as a dictionaty

15.0

One can also read as a dictionary with key-value pairs: example relative mat alignment parameter "LocM100"

In [37]:
#geo.snd_geo['Scifi'].keys()
for key in geo.snd_geo['Scifi'].keys():
    if 'LocM100' in key:
        print(key)

LocM100
LocM100t_0
LocM100t_4361
LocM100t_4575
LocM100t_4855
LocM100t_5172
LocM100t_5431
LocM100t_6305


Lets compare the values for two different tags:

In [40]:
print(Scifi.GetConfParF('Scifi/LocM100t_4575')/u.mm)
print(Scifi.GetConfParF('Scifi/LocM100t_6305')/u.mm)

0.31644001603126526
0.3748000040650368


Lets get the coordinates of a hit - it means getting the coordinates of the sensitive volume that registerred the hit
For Scifi the function is called GetSiPMPosition, but for MuFilter it is GetPosition.

In [43]:
A, B = ROOT.TVector3(), ROOT.TVector3()
print('before')
for i, aHit in enumerate(tree.Digi_ScifiHits):
    Scifi.GetSiPMPosition(aHit.GetDetectorID(),A,B)
    print(aHit.isVertical())
    A.Print()# left
    B.Print()# right
    if i==0: 
        break # only print the a few hits for simplicity of the output
print('After')
Scifi.InitEvent(tree.EventHeader)# provide event header to the detector object
for i, aHit in enumerate(tree.Digi_ScifiHits):
    Scifi.GetSiPMPosition(aHit.GetDetectorID(),A,B)
    print(aHit.isVertical())
    A.Print()# left
    B.Print()# right
    if i==0: 
        break # only print the a few hits for simplicity of the output

before
False
After
False
TVector3 A 3D physics vector (x,y,z)=(-46.154983,31.243707,300.092594) (rho,theta,phi)=(305.224535,10.521535,145.904738)
TVector3 A 3D physics vector (x,y,z)=(-7.155584,31.462365,300.092109) (rho,theta,phi)=(301.821730,6.136849,102.813004)
TVector3 A 3D physics vector (x,y,z)=(-46.155507,31.243184,300.092587) (rho,theta,phi)=(305.224554,10.521562,145.905485)
TVector3 A 3D physics vector (x,y,z)=(-7.156063,31.455212,300.092020) (rho,theta,phi)=(301.820907,6.135555,102.816652)


Wait a bit... did we forget something here? - remember to provide the eventHeader to the detector object via #detector.InitEvent(#eventHeader)

Any suggestions what else to try?